# OBJETIVO
Este archivo tiene por objetivo crear un csv con los datos relevantes para ser cargados en la base de datos. Además, como fallback,
se crea un script sql con los mismos datos

In [1]:
# READ EXCEL - TRANSFORM TO CSV
import pandas as pd 
import os

file_path = '../data/matriculas_ed_superior_nuble_2021.xlsx'
df = pd.read_excel(file_path)

# check if files exists, if yes, don't overwrite
if os.path.exists('../data/matriculas_ed_superior_nuble_2021.csv'):
    print('File already exists, exiting without overwriting.')
else:
    df.to_csv('../data/matriculas_ed_superior_nuble_2021.csv', index=False)

File already exists, exiting without overwriting.


In [ ]:
# HELPER FUNCTIONS
def get_tipo_institucion_mapping():
    mapping = {}
    institucion_types = df['TIPO DE INSTITUCION'].dropna().unique()
    for i, tipo in enumerate(sorted(institucion_types)):
        mapping[tipo] = i + 1 
    return mapping

# For TIPOS_ACREDITACION
def get_tipo_acreditacion_mapping():
    mapping = {}
    acreditacion_types = df['ACREDITACION INSTITUCIONAL'].dropna().unique()
    for i, tipo in enumerate(sorted(acreditacion_types)):
        mapping[tipo] = i + 1
    return mapping

# extract periodos de acreditacion (begin , end) date
def extract_periodo(periodo_str):
    if pd.isna(periodo_str):
        return (None, None)
    try:
        start_str, end_str = periodo_str.split(' AL ')
        start_date = pd.to_datetime(start_str, format='%d/%m/%Y', errors='coerce')
        end_date = pd.to_datetime(end_str, format='%d/%m/%Y', errors='coerce')

        start_formatted = start_date.strftime('%d/%m/%Y') if pd.notna(start_date) else None
        end_formatted = end_date.strftime('%d/%m/%Y') if pd.notna(end_date) else None
        return (start_formatted, end_formatted)
    except Exception as e:
        return (None, None)

In [30]:
tipo_institucion_map = get_tipo_institucion_mapping()
tipo_acreditacion_map = get_tipo_acreditacion_mapping()

institucion_df = df[['TIPO DE INSTITUCION', 'NOMBRE DE INSTITUCION', 'ACREDITACION INSTITUCIONAL', 'PERIODO DE ACREDITACION']].copy()
institucion_df['TIPO DE INSTITUCION'] = institucion_df['TIPO DE INSTITUCION'].map(tipo_institucion_map)
institucion_df['ACREDITACION INSTITUCIONAL'] = institucion_df['ACREDITACION INSTITUCIONAL'].map(tipo_acreditacion_map)
institucion_df[['PERIODO_ACREDITACION_INICIO', 'PERIODO_ACREDITACION_FIN']] = institucion_df['PERIODO DE ACREDITACION'].apply(lambda x: pd.Series(extract_periodo(x)))

print("Unmapped institution types:", institucion_df[institucion_df['TIPO DE INSTITUCION'].isna()]['TIPO DE INSTITUCION'].unique())
print("Unmapped accreditation types:", institucion_df[institucion_df['ACREDITACION INSTITUCIONAL'].isna()]['ACREDITACION INSTITUCIONAL'].unique())
print("Mapping for institutions:", tipo_institucion_map)
print("Mapping for accreditation:", tipo_acreditacion_map)

Unmapped institution types: []
Unmapped accreditation types: []


In [34]:
instituciones_for_db = institucion_df.copy()
instituciones_for_db = instituciones_for_db.rename(columns={
    'NOMBRE DE INSTITUCION': 'institucion_nombre',
    'TIPO DE INSTITUCION': 'tipo_institucion_id',
    'ACREDITACION INSTITUCIONAL': 'acreditacion_id',
    'PERIODO_ACREDITACION_INICIO': 'periodo_acreditacion_inicio',
    'PERIODO_ACREDITACION_FIN': 'periodo_acreditacion_fin'
})

# Select only the columns needed for the database table
instituciones_for_db = instituciones_for_db[['institucion_nombre', 'tipo_institucion_id', 'acreditacion_id', 'periodo_acreditacion_inicio', 'periodo_acreditacion_fin']]

instituciones_for_db = instituciones_for_db.drop_duplicates().reset_index(drop=True)
os.makedirs('../seeds', exist_ok=True)

instituciones_for_db.to_csv('../seeds/instituciones.csv', index=False, na_rep='NULL')

print(f"Created instituciones.csv with {len(instituciones_for_db)} unique institutions")
print("Columns:", list(instituciones_for_db.columns))
print("\nFirst 5 rows:")
print(instituciones_for_db.head())

Created instituciones.csv with 15 unique institutions
Columns: ['institucion_nombre', 'tipo_institucion_id', 'acreditacion_id', 'periodo_acreditacion_inicio', 'periodo_acreditacion_fin']

First 5 rows:
                                institucion_nombre  tipo_institucion_id  \
0                                       CFT INACAP                    1   
1                          UNIVERSIDAD DEL BIO-BIO                    3   
2                                   IP SANTO TOMAS                    2   
3  UNIVERSIDAD CATOLICA DE LA SANTISIMA CONCEPCION                    3   
4                        UNIVERSIDAD DE CONCEPCION                    3   

   acreditacion_id periodo_acreditacion_inicio periodo_acreditacion_fin  
0                1                  05/01/2018               05/01/2025  
1                1                  28/08/2019               28/08/2024  
2                1                  20/01/2021               20/01/2026  
3                1                  03/03/2021     

In [35]:
# CREATE ORACLE SQL INSERT SCRIPT FOR INSTITUCIONES TABLE
def create_instituciones_sql_script(df, filename='../seeds/instituciones_inserts.sql'):
    """
    Creates an Oracle SQL script with INSERT statements for the instituciones table
    """
    sql_lines = []
    sql_lines.append("-- INSERT statements for INSTITUCIONES table")
    sql_lines.append("-- Generated automatically from ETL process")
    sql_lines.append("")
    
    for index, row in df.iterrows():
        # Handle NULL values and format strings properly for Oracle
        institucion_nombre = f"'{row['institucion_nombre'].replace("'", "''")}'" if pd.notna(row['institucion_nombre']) else 'NULL'
        tipo_institucion_id = str(int(row['tipo_institucion_id'])) if pd.notna(row['tipo_institucion_id']) else 'NULL'
        acreditacion_id = str(int(row['acreditacion_id'])) if pd.notna(row['acreditacion_id']) else 'NULL'
        
        # Handle dates - use TO_DATE function for Oracle
        if pd.notna(row['periodo_acreditacion_inicio']) and row['periodo_acreditacion_inicio'] != 'NULL':
            periodo_inicio = f"TO_DATE('{row['periodo_acreditacion_inicio']}', 'DD/MM/YYYY')"
        else:
            periodo_inicio = 'NULL'
            
        if pd.notna(row['periodo_acreditacion_fin']) and row['periodo_acreditacion_fin'] != 'NULL':
            periodo_fin = f"TO_DATE('{row['periodo_acreditacion_fin']}', 'DD/MM/YYYY')"
        else:
            periodo_fin = 'NULL'
        
        # Create INSERT statement
        insert_stmt = f"""INSERT INTO instituciones (institucion_nombre, tipo_institucion_id, acreditacion_id, periodo_acreditacion_inicio, periodo_acreditacion_fin) 
VALUES ({institucion_nombre}, {tipo_institucion_id}, {acreditacion_id}, {periodo_inicio}, {periodo_fin});"""
        
        sql_lines.append(insert_stmt)
    
    sql_lines.append("")
    sql_lines.append("COMMIT;")
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write('\n'.join(sql_lines))
    
    return len(df)

# Create the SQL script
num_inserts = create_instituciones_sql_script(instituciones_for_db)
print(f"Created instituciones_inserts.sql with {num_inserts} INSERT statements")
print("File saved to: ../seeds/instituciones_inserts.sql")

# Show first few lines of the generated SQL
with open('../seeds/instituciones_inserts.sql', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    print("\nFirst 10 lines of generated SQL:")
    for i, line in enumerate(lines[:10]):
        print(f"{i+1:2d}: {line.rstrip()}")

Created instituciones_inserts.sql with 15 INSERT statements
File saved to: ../seeds/instituciones_inserts.sql

First 10 lines of generated SQL:
 1: -- INSERT statements for INSTITUCIONES table
 2: -- Generated automatically from ETL process
 3: 
 4: INSERT INTO instituciones (institucion_nombre, tipo_institucion_id, acreditacion_id, periodo_acreditacion_inicio, periodo_acreditacion_fin)
 5: VALUES ('CFT INACAP', 1, 1, TO_DATE('05/01/2018', 'DD/MM/YYYY'), TO_DATE('05/01/2025', 'DD/MM/YYYY'));
 6: INSERT INTO instituciones (institucion_nombre, tipo_institucion_id, acreditacion_id, periodo_acreditacion_inicio, periodo_acreditacion_fin)
 7: VALUES ('UNIVERSIDAD DEL BIO-BIO', 3, 1, TO_DATE('28/08/2019', 'DD/MM/YYYY'), TO_DATE('28/08/2024', 'DD/MM/YYYY'));
 8: INSERT INTO instituciones (institucion_nombre, tipo_institucion_id, acreditacion_id, periodo_acreditacion_inicio, periodo_acreditacion_fin)
 9: VALUES ('IP SANTO TOMAS', 2, 1, TO_DATE('20/01/2021', 'DD/MM/YYYY'), TO_DATE('20/01/2026', 